In [1]:

import torch
import torchvision
from torchvision.transforms import functional as F
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.utils.data import Dataset
from os import listdir
from os.path import isfile, join
import json
from PIL import Image
import numpy as np
from torchvision.io import read_image
from torchvision import tv_tensors
from torchvision.transforms.v2 import functional as F
import torch.nn.functional as FF
import torch.nn as nn
from torch import Tensor

In [2]:
from torchvision.transforms import v2 as T
# from google.colab import drive
import os

# drive.mount('/content/drive')
# os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/engine.py")
# os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/utils.py")
# os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/coco_utils.py")
# os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/coco_eval.py")
# os.system("wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/transforms.py")


def get_transform(train):
    transforms = []
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    transforms.append(T.ToDtype(torch.float, scale=True))
    transforms.append(T.ToPureTensor())
    return T.Compose(transforms)

In [3]:
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
import torchvision.transforms as transforms
import pandas as pd
from torchvision import tv_tensors
from torchvision.io import read_image

class CustomImageDataset(Dataset):
    def __init__(self, image_dir, labels, transform=None):
        self.image_dir = image_dir
        self.labels = labels
        self.transform = transform
        self.image_foldernames = list(self.labels["id"])
#         print(self.image_foldernames)

    def __len__(self):
        return len(self.image_foldernames)

    def __getitem__(self, idx):
        tmplistofImages = []
        for filename in os.listdir(self.image_dir+"/"+str(self.image_foldernames[idx])):
            if filename.endswith(('.png', '.jpg', '.jpeg')):
                img_path = os.path.join(self.image_dir+"/"+str(self.image_foldernames[idx]), filename)
                img = read_image(img_path)
                image = tv_tensors.Image(img).to(torch.float32)
                if self.transform:
                    image = self.transform(image)
                    tmplistofImages.append(image)
                    
        if len(tmplistofImages)<10:
            for i in range(10-len(tmplistofImages)):
                tmplistofImages.append(image)
                
        stacked_images = torch.cat(tmplistofImages, dim=0)
        # Get the 4-class labels for this image
        row = self.labels.iloc[idx]
        labels = torch.tensor(row["pool"], dtype=torch.int64)
        return stacked_images, labels


class CustomImageDatasettest(Dataset):
    def __init__(self, image_dir, labels, transform=None):
        self.image_dir = image_dir
        self.labels = labels
        self.transform = transform
        self.image_foldernames = list(self.labels["id"])
#         print("\n\n",self.labels["id"])

    def __len__(self):
        return len(self.image_foldernames)

    def __getitem__(self, idx):
        tmplistofImages = []
        for filename in os.listdir(self.image_dir+"/"+str(self.image_foldernames[idx])):
            if filename.endswith(('.png', '.jpg', '.jpeg')):
                img_path = os.path.join(self.image_dir+"/"+str(self.image_foldernames[idx]), filename)
                img = read_image(img_path)
                image = tv_tensors.Image(img).to(torch.float32)
                if self.transform:
                    image = self.transform(image)
                    tmplistofImages.append(image)
                    
        if len(tmplistofImages)<10:
            for i in range(10-len(tmplistofImages)):
                tmplistofImages.append(image)
                
        stacked_images = torch.cat(tmplistofImages, dim=0)
        return stacked_images,self.image_foldernames[idx]


# Example transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
])

def read_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

def check_phrases(x, phrase_a, phrase_b):
    # Check if either phrase_a or phrase_b is in the string x
    if phrase_a in x or phrase_b in x:
        return 1
    else:
        return 0
    
labels = read_json('C:/Users/ASUS/QueraContest/Q4/train.json')

trainLabels = pd.DataFrame(columns=['id','pool'])
for i in range(len(labels["rooms"])):
    tmpdict = dict(labels["rooms"][i])
    trainLabels.loc[len(trainLabels)] = [tmpdict["id"],check_phrases(tmpdict["description"], "استخر", "استخردار")]
    
# print(trainLabels)  
dataset = CustomImageDataset(image_dir='C:/Users/ASUS/QueraContest/Q4/train', labels=trainLabels, transform=transform)

from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Subset

dataset_size = len(dataset)
indices = list(range(dataset_size))
train_indices, test_indices = train_test_split(indices, test_size=0.001, random_state=42)

train_dataset = Subset(dataset, train_indices)
test_dataset = Subset(dataset, test_indices)

labels = read_json('C:/Users/ASUS/QueraContest/Q4/test.json')
testLabels = pd.DataFrame(columns=['id'])
for i in range(len(labels["rooms"])):
    tmpdict = labels["rooms"][i]
    testLabels.loc[len(testLabels)] = tmpdict["id"]
    
# print(testLabels) 
dataset_testfinal = CustomImageDatasettest(image_dir='C:/Users/ASUS/QueraContest/Q4/test', labels=testLabels, transform=transform)

dataloader = DataLoader(train_dataset, batch_size=5, shuffle=True)
dataloader_test = DataLoader(test_dataset, batch_size=1, shuffle=True)
dataloader_test_final = DataLoader(dataset_testfinal, batch_size=1, shuffle=False)

In [4]:
import torch
import torch.nn as nn
import torchvision.models as models

class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion * planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion * planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        out = FF.relu(self.bn1(self.conv1(x)))
        out = FF.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = FF.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=2):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(30, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        x = FF.relu(self.bn1(self.conv1(x)))
        x = FF.max_pool2d(x, kernel_size=3, stride=2, padding=1)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x.view(x.size(0), 1, 2)

def MultiOutputResNet101():
    return ResNet(Bottleneck, [30, 1, 23, 2])

model = MultiOutputResNet101()

In [5]:
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = optim.Adam(model.parameters(), lr=0.0003)
criterion = nn.CrossEntropyLoss()

num_epochs = 8
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    counter =0
    for inputs, info in dataloader:
        inputs, info = inputs.to(device), info.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs[:,0,:], info)
        print(loss)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        counter+=1
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/counter:.4f}")

print("Training complete!")

In [6]:
import torch.optim as optim

counter = 0
num_epochs = 1
for epoch in range(num_epochs):
    model.eval()
    correct_predictions = 0.0
    for inputs, info in dataloader_test:
        inputs, info = inputs.to(device), info.to(device)
        # Forward pass
        outputs = model(inputs)
        _, predicted = torch.max(outputs[:, 0, :], 1)
        print(counter)
        correct_predictions += (predicted == info).sum().item()
        counter+=1

In [7]:
(correct_predictions/counter)

In [8]:
import torch.optim as optim

df = pd.DataFrame(columns=['id','Pool'])
counter = 0
num_epochs = 1
for epoch in range(num_epochs):
    model.eval()
    correct_predictions = 0.0
    for inputs,imageid in dataloader_test_final:
        inputs,imageid = inputs.to(device),imageid.to(device)
        # Forward pass
        outputs = model(inputs)
        _, predicted = torch.max(outputs[:, 0, :], 1)
        counter+=1
        df.loc[len(df)] = [int(imageid.cpu()),int(predicted.cpu())]

In [9]:
finaldf = pd.DataFrame({'id': df['id'], 'pool': df['Pool']==1})
finaldf